# Libraries Used

In [2]:
# Libraries needed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [3]:
df=pd.read_pickle("WM811K.pkl")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 811457 entries, 0 to 811456
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   dieSize         811457 non-null  float64
 1   failureType     811457 non-null  object 
 2   lotName         811457 non-null  object 
 3   trainTestLabel  811457 non-null  object 
 4   waferIndex      811457 non-null  float64
 5   waferMap        811457 non-null  object 
dtypes: float64(2), object(4)
memory usage: 37.1+ MB


In [4]:
df.head()

,dieSize,failureType,lotName,trainTestLabel,waferIndex,waferMap
0,1683.0,none,lot1,Training,1.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
1,1683.0,none,lot1,Training,2.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
2,1683.0,none,lot1,Training,3.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
3,1683.0,none,lot1,Training,4.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
4,1683.0,none,lot1,Training,5.0,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [5]:
max_col = 0
max_row = 0
min_col = 999
min_row = 999

max_val = 0
min_val = 99999999
for idx, data in enumerate(df['waferMap']):
    size = data.shape
    if max_row < size[0]: 
        max_row = size[0]
        max_row_idx = idx
    if max_col < size[1]:
        max_col = size[1]
        max_col_idx = idx
    if min_row > size[0]:
        min_row = size[0]
        min_row_idx = idx
    if min_col > size[1]:
        min_col = size[1]
        min_col_idx = idx

    # if idx == 1: print(data)
    temp_max = np.max(data)
    temp_min = np.min(data)

    # print(temp_min, temp_max)

    if max_val < temp_max: max_val = temp_max
    if min_val > temp_min: min_val = temp_min

print(max_row, max_row_idx)
print(max_col, max_col_idx)
print(min_row, min_row_idx)
print(min_col, min_col_idx)

print(min_val, max_val)

300 244100
205 283434
6 742367
3 745584
0 2


In [6]:
print(df.iloc[max_row_idx,:]['waferMap'].shape)
print(df.iloc[max_col_idx,:]['waferMap'].shape)
print(df.iloc[min_row_idx,:]['waferMap'].shape)
print(df.iloc[min_col_idx,:]['waferMap'].shape)

(300, 202)
(136, 205)
(6, 21)
(15, 3)


# Activation function

In [7]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softplus(x):
    return np.log10(1+np.exp(x))

def diff_softplus(x):
    return sigmoid(x)

# Additional Function

In [ ]:
layer = [9,2,9]; len_layers = len(layer)
in_layer = np.empty(len(layer), dtype=object)

def fill_array_weight(layer, len_layers):
    arr = np.empty(len_layers, dtype=object)

    for i in range(len_layers//2 + 1):
        arr[i] = np.full((layer[i],layer[i+1]),0.5)
    for i in range(len_layers//2 + 1, len_layers):
        arr[i] = np.full((layer[i],layer[i-1]),0.5)
    return arr

def fill_array_bias(layer, len_layers):
    arr = np.empty(len_layers, dtype=object)

    for i in range(len_layers):
        arr[i] = np.full((layer[i],1),0.5)
    return arr

def fill_node(layer, len_layers):
    arr = np.empty(len_layers, dtype=object)

    for i in range(len_layers):
        arr[i] = np.full((layer[i],1),0.5)
    return arr


# testing_weight = fill_array_weight(layer, len_layers)
# testing_bias = fill_array_bias(layer, len_layers)
# print(testing_weight)
# print(testing_bias)

# Forward VAE

In [12]:
def testing(kiwi, eagle):
    eagle[0] = np.sqrt(kiwi)
    eagle[1] = softplus(eagle[0])
    return eagle

nums = np.array([i**2 for i in range(1,11)])
eagle = np.zeros((2,10))
eagle = testing(nums,eagle)
print(eagle)

NameError: name 'softplus' is not defined

In [12]:
def forward(pixel, wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight, in_layer, len_layers, aLeft_mean, aLeft_var, zLeft_mean, zLeft_var, aRight, zRight):
    in_layer[0] = pixel
    for layer in range(len_layers // 2):
        zLeft_mean[layer + 1] = np.matmul(wLeft_mean[layer + 1], in_layer[layer]) + bLeft_mean[layer + 1]
        zLeft_var[layer + 1] = np.matmul(wLeft_var[layer + 1], in_layer[layer]) + bLeft_var[layer + 1]

        aLeft_mean[layer + 1] = zLeft_mean[layer + 1]
        aLeft_var[layer + 1] = softplus(zLeft_var[layer + 1])

        eps = np.random.normal(size=zLeft_mean[layer + 1].shape)
        in_layer[layer + 1] = aLeft_mean[layer + 1] + np.sqrt(aLeft_var[layer + 1]) * eps

    for layer in range(len_layers // 2, len_layers - 1):
        # zRight[layer + 1] = np.matmul(wRight[layer + 1 - len_layers // 2], in_layer[layer]) + bRight[layer + 1 - len_layers // 2]
        zRight[layer+1] = np.matmul(wRight[layer+1],in_layer[layer]) + bRight[layer+1]
        aRight[layer + 1] = sigmoid(zRight[layer + 1])

        in_layer[layer + 1] = aRight[layer + 1]

In [24]:
# Testing using O data

layer = [9,2,9]; len_layers = len(layer)
in_layer = np.empty(len(layer), dtype=object)
# Populate the 'in_layer' with arrays of varying lengths
for i, size in enumerate(layer):
    in_layer[i] = np.zeros(size)  # Create an array of zeros for each row, with length based on 'layer'

image = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
pixel = image.reshape(9,1)

wLeft_mean  = [[0 for i in range(layer[0])],
               [[0.0151, -0.0046, -0.0178, 0.0006, -0.0177, 0.0122, -0.3224, 2.0527, 0.2468],
                [-0.0042, 0.0363, -0.0033, 0.0364, -0.0387, 0.0360, 0.3951, -0.1630, 0.5387]]]
bLeft_mean  = [[0 for i in range(layer[1])],
               [[-0.8641],
                [-0.8743]]]

wLeft_var   = [[0 for i in range(layer[0])],
               [[0.0567, -0.0224, -0.0221, 0.0101, 0.0237, -0.0150, -0.0886, -0.0722, -0.3455],
                [0.0436, -0.0386, 0.0429, -0.0360, 0.0827, -0.0475, 0.8645, 0.1661, 0.1577]]]
bLeft_var   = [[0 for i in range(layer[1])],
               [[-0.9129],
                [-0.6264]]]

wRight = [[0 for i in range(layer[0])],
          [0 for i in range(layer[1])],
          [[0.1393, 0.2673],
           [4.9796, -0.1095],
           [0.1383, 0.2631],
           [4.9718, -0.1096],
           [-4.9381,0.1126],
           [4.9775, -0.1089],
           [0.1435, 0.2764],
           [4.9814, -0.1098],
           [0.1338, 0.2538]]]

bRight      = [[0 for i in range(layer[0])],
               [0 for i in range(layer[1])],
               [[4.9357],
                [-0.2049],
                [4.9334],
                [-0.2035],
                [0.1958],
                [-0.2049],
                [4.9430],
                [-0.2050],
                [4.9264]]]

aLeft_mean  = fill_node(layer, len_layers)
aLeft_var   = fill_node(layer, len_layers)
zLeft_mean  = fill_node(layer, len_layers)
zLeft_var   = fill_node(layer, len_layers)
aRight      = fill_node(layer, len_layers)
zRight      = fill_node(layer, len_layers)


forward(pixel, wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight, in_layer, len_layers, aLeft_mean, aLeft_var, zLeft_mean, zLeft_var, aRight, zRight)
print(np.round(in_layer[len_layers-1]))



[[1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [1.]
 [1.]
 [1.]
 [1.]]


# Loss Function

In [23]:
def reconstruction_loss(prev_out, label):
    # Finding reconstruction loss using Binary Cross Entropy Error
    err_rec = -label * np.log10(prev_out) - (1-label) * np.log10(1-prev_out)
    return err_rec

def regularization_loss(mean_latent, var_latent):
    # Finding regularization loss using Binary Cross Entropy Error
    err_reg = -0.5 * (1 + np.log10(var_latent) - np.square(mean_latent) - var_latent)
    return err_reg

def total_loss(prev_out, label, mean_latent, var_latent):
    return reconstruction_loss(prev_out,label) + regularization_loss(mean_latent,var_latent)

In [40]:
E = np.empty(0,dtype=object)
E1 = np.full(layer[0],fill_value=1)
E2 = np.full(layer[0],fill_value=2)
print(E2/5)
# np.append(E, E2)
# print(E)
print(layer[0])

[0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4 0.4]
9


In [42]:
def fill_gradient(layer, len_layers):
    arr = np.empty(len_layers, dtype=object)

    for i in range(len_layers):
        arr[i] = np.zeros((layer[i],1))  # Create an array of zeros for each row, with length based on 'layer'
    return arr


# Backward VAE

In [ ]:
def train_vae(pixel,label, wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight, in_layer, len_layers, aLeft_mean, aLeft_var, zLeft_mean, zLeft_var, aRight, zRight, history_loss, epoch, rate):
    in_layer[0] = pixel
    for iter in range(epoch):
        print(f"[INFO] Training Epoch: {iter}")
        # Forward Propagation
        for layer in range (len_layers//2):
            # print(layer)
            zLeft_mean[layer+1] = np.matmul(wLeft_mean[layer+1],in_layer[layer]) + bLeft_mean[layer+1]
            zLeft_var[layer+1] = np.matmul(wLeft_var[layer+1],in_layer[layer]) + bLeft_var[layer+1]

            aLeft_mean[layer+1] = zLeft_mean[layer+1]
            aLeft_var[layer+1] = softplus(zLeft_var[layer+1])

            eps = np.random.normal(size=zLeft_mean[layer+1].shape)
            in_layer[layer+1] = aLeft_mean[layer+1] + np.sqrt(aLeft_var[layer+1]) * eps


        for layer in range(len_layers//2, len_layers-1):
            zRight[layer+1] = np.matmul(wRight[layer+1],in_layer[layer]) + bRight[layer+1]
            aRight[layer+1] = sigmoid(zRight[layer+1])

            in_layer[layer+1] = aRight[layer+1]
        
        # Backward Propagation
        history_loss[iter] = total_loss(in_layer[len_layers-1], label, aLeft_mean[len_layers//2], aLeft_var[len_layers//2]) / in_layer[0]

        for layer in (len_layers-1,len_layers//2,-1):
            # Decoder part (right)
            dEda_right = -label / in_layer[layer] + (1 - label) / (1 - in_layer[layer])
            dEdz_right = dEda_right * in_layer[layer] * (1 - in_layer[layer])
            delta_right = dEdz_right

            dEdb_right = np.sum(delta_right, axis=1)
            dEdw_right = np.matmul(in_layer[layer - 1].T, delta_right)

            # Update decoder (right) weights and biases
            wRight[layer + 1] -= rate * dEdw_right
            bRight[layer + 1] -= rate * dEdb_right

        # Encoder part (left)
        delta_left_mean = delta_right
        delta_left_var = delta_right
        for layer in range(len_layers // 2, 0, -1):  # Loop over the encoder layers
            # For the encoder (left), compute the gradients for mean and variance
            dEda_left_mean = np.matmul(wRight[layer + 1].T, delta_left_mean) + aLeft_mean[layer + 1]
            dEda_left_var = np.matmul(wRight[layer + 1].T, delta_left_var) * eps / (2 * np.sqrt(aLeft_var[layer + 1])) + 0.5 * (1 - 1 / aLeft_var[layer + 1])

            dEdz_left_mean = dEda_left_mean
            dEdz2_left_var = dEda_left_var * (1 / (1 + np.exp(-zLeft_var[layer + 1])))  # Sigmoid derivative of softplus

            delta_left_mean = dEdz_left_mean
            delta_left_var = dEdz2_left_var

            # Compute gradients for the encoder (left) layers
            dEdb2_mean = np.sum(delta_left_mean, axis=1)
            dEdw2_mean = np.matmul(in_layer[layer - 1].T, delta_left_mean)
            dEdb2_var = np.sum(delta_left_var, axis=1)
            dEdw2_var = np.matmul(in_layer[layer - 1].T, delta_left_var)

            # Update encoder (left) weights and biases
            wLeft_mean[layer + 1] -= rate * dEdw2_mean
            bLeft_mean[layer + 1] -= rate * dEdb2_mean
            wLeft_var[layer + 1] -= rate * dEdw2_var
            bLeft_var[layer + 1] -= rate * dEdb2_var
            


In [17]:

def backward(label, wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight, in_layer, len_layers, aLeft_mean, aLeft_var, zLeft_mean, zLeft_var, aRight, zRight, gradients, loss):
    dLoss = loss  # Assume loss is a gradient of the final layer output wrt loss

    # Backpropagation for the decoder (Right side)
    for layer in reversed(range(len_layers // 2, len_layers - 1)):
        dz = dLoss * (aRight[layer + 1] * (1 - aRight[layer + 1]))  # Derivative of sigmoid
        gradients['wRight'][layer - len_layers // 2] = np.matmul(dz, in_layer[layer].T)
        gradients['bRight'][layer - len_layers // 2] = dz
        dLoss = np.matmul(wRight[layer - len_layers // 2].T, dz)

    # Backpropagation for the encoder (Left side)
    for layer in reversed(range(len_layers // 2)):
        dz_mean = dLoss
        dz_var = dLoss * diff_softplus(zLeft_var[layer + 1])

        gradients['wLeft_mean'][layer + 1] = np.matmul(dz_mean, in_layer[layer].T)
        gradients['bLeft_mean'][layer + 1] = dz_mean

        gradients['wLeft_var'][layer + 1] = np.matmul(dz_var, in_layer[layer].T)
        gradients['bLeft_var'][layer + 1] = dz_var

        dLoss = np.matmul(wLeft_mean[layer + 1].T, dz_mean) + np.matmul(wLeft_var[layer + 1].T, dz_var)

# Example initialization of gradients dictionary
def initialize_gradients(len_layers, layer):
    gradients = {
        'wLeft_mean': [np.zeros((layer[i + 1], layer[i])) if i + 1 < len_layers // 2 else None for i in range(len_layers - 1)],
        'bLeft_mean': [np.zeros((layer[i + 1], 1)) if i + 1 < len_layers // 2 else None for i in range(len_layers - 1)],
        'wLeft_var': [np.zeros((layer[i + 1], layer[i])) if i + 1 < len_layers // 2 else None for i in range(len_layers - 1)],
        'bLeft_var': [np.zeros((layer[i + 1], 1)) if i + 1 < len_layers // 2 else None for i in range(len_layers - 1)],
        'wRight': [np.zeros((layer[i], layer[i - 1])) if i >= len_layers // 2 else None for i in range(len_layers - 1)],
        'bRight': [np.zeros((layer[i], 1)) if i >= len_layers // 2 else None for i in range(len_layers - 1)],
    }
    return gradients



In [31]:
# Test case setup
layer = [9, 2, 9]
len_layers = len(layer)
in_layer = np.empty(len_layers, dtype=object)

# Initialize weights and biases
def init_test_params(layer, len_layers):
    wLeft_mean = [None] + [np.random.normal(size=(layer[i + 1], layer[i])) for i in range(len_layers // 2)]
    wLeft_var = [None] + [np.random.normal(size=(layer[i + 1], layer[i])) for i in range(len_layers // 2)]
    bLeft_mean = [None] + [np.random.normal(size=(layer[i + 1], 1)) for i in range(len_layers // 2)]
    bLeft_var = [None] + [np.random.normal(size=(layer[i + 1], 1)) for i in range(len_layers // 2)]

    wRight = [np.random.normal(size=(layer[i + 1], layer[i])) if i >= len_layers // 2 else None for i in range(len_layers - 1)]
    bRight = [np.random.normal(size=(layer[i + 1], 1)) if i >= len_layers // 2 else None for i in range(len_layers - 1)]

    return wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight

wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight = init_test_params(layer, len_layers)

aLeft_mean = [np.zeros((size, 1)) for size in layer]
aLeft_var = [np.zeros((size, 1)) for size in layer]
zLeft_mean = [np.zeros((size, 1)) for size in layer]
zLeft_var = [np.zeros((size, 1)) for size in layer]
aRight = [np.zeros((size, 1)) for size in layer]
zRight = [np.zeros((size, 1)) for size in layer]

# Create test input
image = np.array([[1, 1, 1], [1, 0, 1], [1, 1, 1]])
pixel = image.reshape(9, 1)

# Initialize gradients
gradients = initialize_gradients(len_layers, layer)

# Forward pass
forward(pixel, wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight, in_layer, len_layers, aLeft_mean, aLeft_var, zLeft_mean, zLeft_var, aRight, zRight)

# Simulated loss gradient from output layer
loss_gradient = np.random.normal(size=aRight[-1].shape)

# Backward pass
backward(pixel, wLeft_mean, wLeft_var, bLeft_mean, bLeft_var, wRight, bRight, in_layer, len_layers, aLeft_mean, aLeft_var, zLeft_mean, zLeft_var, aRight, zRight, gradients, loss_gradient)

# Print results
print("Forward Pass Output:")
print(aRight[-1])
print("\nGradient Results:")
print(gradients)

IndexError: list assignment index out of range